In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
from nuclass.data import icecat1_cut
import numpy as np
import pandas as pd
from tqdm import tqdm
from astropy.time import Time
from astropy.coordinates import SkyCoord
from astropy import units as u

In [ ]:
# window for source analysis, in days, relative to a reference time 
phase_min = 0.
phase_max = +180.

# Set up IceCube data

In [ ]:
# Set Up Times
t_min = icecat1_cut["EVENTMJD"].min()
t_length = icecat1_cut["EVENTMJD"].max() - t_min

We need to convert the IceCube coordinates, which has a center and +/- bounds, to exact corners of a rectangle

In [ ]:
def calculate_recatangle_bounds(df: pd.DataFrame) -> pd.DataFrame:
    """
    Calculate the bounds of a rectangle for a neutrino.

    This is a bit non-trivial, because you need to wrap around the 0/2 pi boundary
    """
    df = df.copy()

    # Calculate bounds of rectangle for each neutrino

    # RA lower bound
    ra_min = df["RARAD"] - np.radians(df["RA_ERR_MINUS"])
    # Wrap around 2 pi
    ra_min[ra_min < 0.] += 2 * np.pi
    df["RAMIN"] = ra_min
    
    # Ra upper bound
    ra_max = df["RARAD"] + np.radians(icecat1_cut["RA_ERR_PLUS"])
    # Wrap around 2 pi
    ra_max[ra_max > 2 * np.pi] -= 2 * np.pi
    df["RAMAX"] = ra_max

    # Dec bounds (here it's fine maths-wise to have bounds beyond +90 or -90)
    df["DECMIN"] = df["DECRAD"] - np.radians(df["DEC_ERR_MINUS"])
    df["DECMAX"] = df["DECRAD"] + np.radians(df["DEC_ERR_PLUS"])
    return df

In [ ]:
icecat1_cut["RARAD"] = np.radians(icecat1_cut["RA"])
icecat1_cut["DECRAD"] = np.radians(icecat1_cut["DEC"])
icecat1_cut = calculate_recatangle_bounds(icecat1_cut)

In [ ]:
def scramble_positions(df: pd.DataFrame) -> pd.DataFrame:
    """
    This function randomly redistributes the neutrino alerts in RA and time, 
    producing a simulated neutrino catalogue
    """
    new_df = df.copy()
    
    # Scramble 
    # Uniform in RA
    new_ra  = np.random.uniform(size=len(df)) * 2 * np.pi
    # Uniform in time
    new_times = t_min + np.random.uniform(size=len(df)) * t_length

    new_df["RARAD"] = new_ra
    new_df["DECRAD"] = np.radians(df["DEC"]) # Keep dec the same
    new_df["EVENTMJD"] = new_times

    new_df = calculate_recatangle_bounds(new_df)
    return new_df

In [ ]:
# Let's try scrambling the catalogue
df = scramble_positions(icecat1_cut)

In [ ]:
# Here we see that all the events have new RA values and new times
df

# Set up the Source Catalogue that you will test

In [ ]:
# This is a completely randomised catalogue of 100 sources, distributed across the sky
n = 100

cat = pd.DataFrame()
cat["RARAD"] = np.random.uniform(size=n) * 2 * np.pi
cat["DECRAD"] = -np.pi/2. + np.random.uniform(size=n) * np.pi
cat["REFMJD"] = t_min + np.random.uniform(size=n) * t_length
cat

# Exercise - replace with a real catalogue here!!!

# Check the number of matches between the neurtrinos and the sources

In [1]:
def check_matches_single(nu: pd.DataFrame, src: pd.DataFrame) -> int:
    """
    Check how many (if any) sources are coincident with a neutrino

    :param nu: pd.Series entry for neutrino, from IceCat
    :param src: Source Catalogue

    :return: Number of matches for one neutrino
    """
    # Check wrap if rectangle overlaps 360 wraparound 
    if nu["RAMAX"] < nu["RAMIN"]:
        # Check if sources lie in the lower or upper RA range
        ra_mask = (
            (src["RARAD"] < nu["RAMAX"]) | (src["RARAD"] > nu["RAMIN"])
        )

    else:
        # Check if source lies in the ra range
        ra_mask = src["RARAD"].between(nu["RAMIN"], nu["RAMAX"])

    # Check if source lies in dec range
    dec_mask = src["DECRAD"].between(nu["DECMIN"], nu["DECMAX"])

    # Check if neutrino is temporally coincident with source
    time_mask = (nu["EVENTMJD"] - src["REFMJD"]).between(phase_min, phase_max)

    # Check if spatially AND temporally coincident
    mask = ra_mask & dec_mask & time_mask
    return int(mask.sum())

def check_matches(src: pd.DataFrame, nudf: pd.DataFrame) -> int:
    """
    Iteratively the total number of neutrino-source matches

    :param src: Source catalogue
    :param nudf: Dataframe containing all neutrino positions
    
    :return: Total number of neutrino-source matches
    """
    n_match = 0
    for i, nu in nudf.iterrows():
        n_match += check_matches_single(nu, src)
    return n_match

NameError: name 'pd' is not defined

In [2]:
true_matches = check_matches(cat, icecat1_cut)
print(f"Found {true_matches} neutrino-catalogue matches")
true_matches

NameError: name 'check_matches' is not defined

# Estimate how likely the result is to arise by chance

In [ ]:
# Calculate background matches
trials = []

In [ ]:
# Run many randomised experiments to compare
n_trial = 1000

for i in tqdm(range(n_trial)):
    df = scramble_positions(icecat1_cut)
    trial_matches = check_matches(cat, df)
    trials.append(trial_matches)


In [ ]:
# Plot the distrubtion of random matches expected, alongside the true result 
data = pd.Series(trials)
left_of_first_bin = data.min() - 0.5
right_of_last_bin = data.max() + 0.5

weight = np.ones(len(data))/len(data)

plt.hist(data, np.arange(left_of_first_bin, right_of_last_bin + 1, 1), weights=weight, edgecolor='black', label="Background PDF")
plt.axvline(true_matches, label="Observed matches", color="k", linestyle="--")
plt.legend()
plt.ylim(top=1.0)

In [ ]:
trial_series = pd.Series(trials)
n_less = (trial_series < true_matches).sum()
n_tot = len(trial_series)
frac = n_less/n_tot

print(f"{n_less} of the {n_tot} trials yielded fewer matches than the real data ({true_matches} matches)")
print(f"This corresponds to {100.*frac:.2f}%, and means an excess above background expectations {['has', 'has not'][int(frac < 0.5)]} been observed")

In [ ]:
# Now, your challenge: find a catalogue of real sources and repeat the test!